<h1><font color="#113D68" size=5>Deep Learning para Procesamiento del Lenguaje Natural</font></h1>



<h1><font color="#113D68" size=6>Desarrollar un Embedding + Modelo CNN para análisis de reseñas</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Conjunto de datos de reseñas de películas](#section1)
* [2. Preparación de datos](#section2)
    * [2.1. Revisiones de carga y limpieza](#section21)
    * [2.2. Definir un vocabulario](#section22)
* [3. Entrenar CNN con capa Embedding](#section3)
* [4. Evaluar modelo](#section4)
* [5. Desarrollar un modelo CNN para n-gramas](#section5)
    * [5.1. Codificar datos](#section51)
    * [5.2. Definir el modelo](#section52)
    * [5.3. Mostrar los resultados](#section53)
    * [5.4. Evaluar el modelo](#section54)
* [6. Extensiones](#section6)
    * [6.1. Extensiones sin n-gramas](#section61)
    * [6.2. Extensiones con n-gramas](#section62)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

Las incrustaciones de palabras son una técnica para representar texto donde diferentes palabras con significado similar tienen una representación vectorial de valor real similar. Son un avance clave que ha llevado a un gran rendimiento de los modelos de redes neuronales en un conjunto de problemas desafiantes de procesamiento del lenguaje natural. En este tutorial, descubrirá cómo desarrollar modelos de incrustación de palabras con redes neuronales convolucionales para clasificar reseñas de películas. Después de completar este tutorial, sabrás:
- Cómo preparar datos de texto de reseñas de películas para su clasificación con métodos de aprendizaje profundo.
- Cómo desarrollar un modelo de clasificación neuronal con incrustación de palabras y capas convolucionales.
- Cómo evaluar el modelo de clasificación neuronal desarrollado.

Este tutorial se divide en las siguientes partes:
1. Conjunto de datos de reseñas de películas
2. Preparación de datos
3. Entrene CNN con capa de incrustación
4. Evaluar modelo

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Conjunto de datos de reseñas de películas</font>

En este tutorial, utilizaremos el conjunto de datos de revisión de películas.

Después de descomprimir el archivo, tendrá un directorio llamado `txt_sentoken` con dos subdirectorios que contienen el texto `neg` y `pos` para reseñas negativas y positivas. Las revisiones se almacenan una por archivo con una convención de nomenclatura `cv000` a `cv999` para cada `neg` y `pos`.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Preparación de datos</font>

Nota: La preparación del conjunto de datos de reseñas de películas se describió por primera vez en capítulos anteriores. En esta sección, veremos 3 cosas:
1. Cargar y limpiar los datos para eliminar puntuación y números.
2. Definición de un vocabulario de palabras preferidas.

<a id="section21"></a>
# <font color="#004D7F" size=5>2.1. Revisiones de carga y limpieza</font>

Los datos de texto ya están bastante limpios; no se requiere mucha preparación. Sin atascarnos demasiado en los detalles, prepararemos los datos de la siguiente manera:
- Fichas divididas en espacios en blanco.
- Eliminar toda la puntuación de las palabras.
- Eliminar todas las palabras que no estén compuestas únicamente por caracteres alfabéticos.
- Eliminar todas las palabras que son palabras vacías conocidas.
- Eliminar todas las palabras que tengan una longitud $\leq$ 1 carácter.

Podemos poner todos estos pasos en una función llamada `clean_doc()` que toma como argumento el texto sin formato cargado desde un archivo y devuelve una lista de tokens limpios. También podemos definir una función `load_doc()` que cargue un documento desde un archivo listo para usar con la función `clean_doc()`. A continuación se muestra un ejemplo de limpieza de la primera crítica positiva.

In [3]:
from nltk.corpus import stopwords
import string
import re

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# prepare regex for char filtering
	re_punc = re.compile('[%s]' % re.escape(string.punctuation))
	# remove punctuation from each word
	tokens = [re_punc.sub('', w) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

# load the document
filename = 'data/txt_sentoken/pos/cv000_29590.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

['films', 'adapted', 'comic', 'books', 'plenty', 'success', 'whether', 'theyre', 'superheroes', 'batman', 'superman', 'spawn', 'geared', 'toward', 'kids', 'casper', 'arthouse', 'crowd', 'ghost', 'world', 'theres', 'never', 'really', 'comic', 'book', 'like', 'hell', 'starters', 'created', 'alan', 'moore', 'eddie', 'campbell', 'brought', 'medium', 'whole', 'new', 'level', 'mid', 'series', 'called', 'watchmen', 'say', 'moore', 'campbell', 'thoroughly', 'researched', 'subject', 'jack', 'ripper', 'would', 'like', 'saying', 'michael', 'jackson', 'starting', 'look', 'little', 'odd', 'book', 'graphic', 'novel', 'pages', 'long', 'includes', 'nearly', 'consist', 'nothing', 'footnotes', 'words', 'dont', 'dismiss', 'film', 'source', 'get', 'past', 'whole', 'comic', 'book', 'thing', 'might', 'find', 'another', 'stumbling', 'block', 'hells', 'directors', 'albert', 'allen', 'hughes', 'getting', 'hughes', 'brothers', 'direct', 'seems', 'almost', 'ludicrous', 'casting', 'carrot', 'top', 'well', 'anythi

Ejecutar el ejemplo imprime una larga lista de tokens limpios. Hay muchos más pasos de limpieza que queramos explorar y los dejo como ejercicios adicionales.

<a id="section22"></a>
# <font color="#004D7F" size=5>2.2. Definir un vocabulario</font>

Podemos desarrollar un vocabulario a modo de `Counter`, que es un diccionario de mapeo de palabras y su conteo que nos permite actualizar y consultar fácilmente. Cada documento se puede agregar a `Counter` (una nueva función llamada `add_doc_to_vocab()`) y podemos pasar por alto todas las revisiones en el directorio negativo y luego en el directorio positivo (una nueva función llamada `process_docs()`). 

- Muestra que tenemos un vocabulario de 44,276 palabras. 
- También podemos ver una muestra de las 50 palabras más utilizadas en las reseñas de películas.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el objeto [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter)

In [4]:
from os import listdir
from collections import Counter

# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
	# load doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# update counts
	vocab.update(tokens)

# load all docs in a directory
def process_docs(directory, vocab):
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# add doc to vocab
		add_doc_to_vocab(path, vocab)

# define vocab
vocab = Counter()
# add all docs to vocab
process_docs('data/txt_sentoken/pos', vocab)
process_docs('data/txt_sentoken/neg', vocab)
# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

44276
[('film', 7983), ('one', 4946), ('movie', 4826), ('like', 3201), ('even', 2262), ('good', 2080), ('time', 2041), ('story', 1907), ('films', 1873), ('would', 1844), ('much', 1824), ('also', 1757), ('characters', 1735), ('get', 1724), ('character', 1703), ('two', 1643), ('first', 1588), ('see', 1557), ('way', 1515), ('well', 1511), ('make', 1418), ('really', 1407), ('little', 1351), ('life', 1334), ('plot', 1288), ('people', 1269), ('bad', 1248), ('could', 1248), ('scene', 1241), ('movies', 1238), ('never', 1201), ('best', 1179), ('new', 1140), ('scenes', 1135), ('man', 1131), ('many', 1130), ('doesnt', 1118), ('know', 1092), ('dont', 1086), ('hes', 1024), ('great', 1014), ('another', 992), ('action', 985), ('love', 977), ('us', 967), ('go', 952), ('director', 948), ('end', 946), ('something', 945), ('still', 936)]


Podemos recorrer el vocabulario y eliminar todas las palabras que tienen poca ocurrencia, como que solo se usan una o dos veces en todas las revisiones. 
- Por ejemplo, el siguiente fragmento recuperará solo los tokens que aparecen 2 o más veces en todas las revisiones.
- Con esta adición muestra que el tamaño del vocabulario se reduce en un poco más de la mitad de su tamaño, de 44.276 a 25.767 palabras.

In [5]:
# keep tokens with a min occurrence
min_occurrence = 2
tokens = [k for k,c in vocab.items() if c >= min_occurrence]
print(len(tokens))

25767


- Guardamos en un nuevo vocabulario en `vocab.txt` que luego podemos cargar y usar para filtrar reseñas de películas antes de codificarlas para el modelado. 
- Definimos una nueva función llamada `save_list()` que guarda el vocabulario en un archivo, con una palabra por línea. Por ejemplo:

In [6]:
# save list to file
def save_list(lines, filename):
	# convert lines to a single blob of text
	data = '\n'.join(lines)
	# open file
	file = open(filename, 'w')
	# write text
	file.write(data)
	# close file
	file.close()

In [7]:
# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')

El orden de las palabras en su archivo será diferente, pero debería verse como el siguiendo:

In [8]:
archivo = open("vocab.txt")
print(archivo.read())

one
fun
activity
parents
holidays
suggest
old
film
see
interest
kids
although
blackandwhite
films
frequently
viewed
suspect
ones
color
greeted
open
mind
find
colorful
action
even
six
decades
ago
real
possibility
take
home
hit
family
wandered
classic
section
local
video
store
day
picked
copy
adventures
robin
hood
high
spirited
version
walter
scott
story
nominated
academy
award
best
picture
winner
three
oscars
wolfgang
melodramatic
music
ralph
dawsons
fast
paced
editing
carl
jules
lush
sets
probably
remembered
errol
charismatic
acting
sir
flynn
handsome
figure
smile
charms
audience
clearly
time
let
cut
chase
say
tape
indeed
popular
rhodes
household
littlest
jeffrey
age
liked
much
least
times
maybe
ill
discuss
fascination
usual
end
review
simply
stated
derives
success
genre
swashbuckler
fights
hundred
men
without
scratch
considered
little
james
bond
production
values
raise
level
rich
poor
steals
gives
every
schoolchild
knows
movie
however
seems
less
interested
income
redistribution
fighti

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Entrenar CNN con capa `Embedding`</font>

- El primer paso es cargar el vocabulario. 
- Lo usaremos para filtrar palabras de reseñas de películas que no nos interesen. 
- Si ha trabajado en la sección anterior, debería tener un archivo local llamado `vocab.txt` con una palabra por línea. 
- Podemos cargar ese archivo y construir un vocabulario como un conjunto para verificar la validez de lo tokens.

In [9]:
import string
import re
from os import listdir
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())

2022-12-05 16:47:54.703558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 16:48:02.512575: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/include:/usr/local/cuda-11.0/lib64:
2022-12-05 16:48:02.513026: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/include:/usr/local/cuda-11.0/lib64:
2022-12-05 16:48

In [10]:
vocab

{'awareness',
 'hc',
 'floom',
 'pseudodocumentary',
 'thanksgiving',
 'invariably',
 'sample',
 'mirandas',
 'firstrun',
 'comedian',
 'inn',
 'revives',
 'abruptly',
 'mcalisters',
 'remnants',
 'modeling',
 'wellearned',
 'cinergi',
 'noun',
 'lights',
 'firstclass',
 'relish',
 'dresses',
 'incomprehensible',
 'colorful',
 'undiscovered',
 'disturbs',
 'gungho',
 'toughtalking',
 'assassin',
 'tripe',
 'udall',
 'slomo',
 'skintight',
 'anne',
 'latter',
 'potatoes',
 'handy',
 'listed',
 'unearthed',
 'subpar',
 'lambs',
 'helming',
 'cunningham',
 'supermodel',
 'favourites',
 'excellently',
 'unfolding',
 'obtaining',
 'overact',
 'proponents',
 'granted',
 'visjnic',
 'blues',
 'attempts',
 'gentle',
 'coincides',
 'suspenseless',
 'stillliving',
 'cup',
 'inconsistent',
 'garfield',
 'generously',
 'taxation',
 'goldstein',
 'strip',
 'renting',
 'mired',
 'examinations',
 'rabies',
 'ranft',
 'shipwreck',
 'affluent',
 'owens',
 'keeble',
 'portals',
 'rables',
 'roads',
 'ne

Limpiamos el documento implica dividir cada revisión en función de los espacios en blanco, eliminar la puntuación y luego filtrar todos los tokens que no están en el vocabulario. 

In [11]:
# turn a doc into clean tokens
def clean_doc(doc, vocab):
	# split into tokens by white space
	tokens = doc.split()
	# prepare regex for char filtering
	re_punc = re.compile('[%s]' % re.escape(string.punctuation))
	# remove punctuation from each word
	tokens = [re_punc.sub('', w) for w in tokens]
	# filter out tokens not in vocab
	tokens = [w for w in tokens if w in vocab]
	tokens = ' '.join(tokens)
	return tokens

A continuación, debemos cargar todas las reseñas de películas de datos de entrenamiento. 

Creamos la función `process_docs()` para cargar los documentos, limpiarlos y devolverlos como una lista de cadenas, con un documento por cadena. Queremos que cada documento sea una cadena para facilitar la codificación como una secuencia de enteros más adelante. 

Actualizamos `process_docs()` para que pueda llamar a la función `clean_doc()` para cada documento en un directorio determinado.

In [12]:
# load all docs in a directory
def process_docs(directory, vocab, is_train):
	documents = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_train and filename.startswith('cv9'):
			continue
		if not is_train and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load the doc
		doc = load_doc(path)
		# clean doc
		tokens = clean_doc(doc, vocab)
		# add to list
		documents.append(tokens)
	return documents

Podemos llamar a la función `process_docs()` para los directorios `neg` y `pos` y combinar las revisiones en un solo train o test. También podemos definir las etiquetas de clase para el conjunto de datos. 

La función `load_clean_dataset()` a continuación cargará todas las revisiones y preparará etiquetas de clase para el conjunto de datos de train/test.

In [13]:
# load and clean a dataset
def load_clean_dataset(vocab, is_train):
	# load documents
	neg = process_docs('data/txt_sentoken/neg', vocab, is_train)
	pos = process_docs('data/txt_sentoken/pos', vocab, is_train)
	docs = neg + pos
	# prepare labels
	labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
	return docs, labels

El siguiente paso es codificar cada documento como una secuencia de números enteros. 
- La capa `Embedding` de Keras requiere entradas de enteros donde cada entero se asigna a un solo token que tiene una representación vectorial de valor real específica dentro de la incrustación. 
- Estos vectores son aleatorios al comienzo del entrenamiento, pero durante el entrenamiento se vuelven significativos para la red. 
- Podemos codificar los documentos de entrenamiento como secuencias de números enteros utilizando la clase `Tokenizer` en la API de Keras. 
    - Primero, debemos construir una instancia de la clase y luego entrenarla en todos los documentos del conjunto de datos de entrenamiento. 
    - En este caso, desarrolla un vocabulario de todos los tokens en el conjunto de datos de entrenamiento y desarrolla un mapeo consistente de palabras en el vocabulario a enteros únicos. 
    - La función `create_tokenizer()` a continuación preparará un `Tokenizer` a partir de los datos de entrenamiento.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre la clase [`Tokenizer`](https://faroit.com/keras-docs/2.0.5/preprocessing/text/#tokenizer)

In [14]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

Ahora que se ha preparado la asignación de palabras a números enteros, podemos usarla para codificar las reseñas en el conjunto de datos de entrenamiento. Podemos hacer eso llamando a la función `texts_to_sequences()` en el `Tokenizer`. También debemos asegurarnos de que todos los documentos tengan la misma longitud. Este es un requisito de Keras para un cálculo eficiente. Podríamos truncar las reseñas al tamaño más pequeño o reseñas de cero (`pad` con el valor 0) a la longitud máxima, o algún híbrido. En este caso, rellenaremos todas las revisiones con la duración de la revisión más larga en el conjunto de datos de entrenamiento. 
- Primero, podemos encontrar la revisión más larga usando la función `max()` en el conjunto de datos de entrenamiento y tomar su longitud.
- A continuación, podemos llamar a la función `pad_Sequences()` de Keras para rellenar las secuencias hasta la longitud máxima añadiendo 0 valores al final.

In [15]:
# load training data
train_docs, ytrain = load_clean_dataset(vocab, True)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)

Maximum length: 1317


Luego podemos usar la longitud máxima como un parámetro para una función para codificar enteros y rellenar las secuencias.

In [16]:
# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
	# integer encode
	encoded = tokenizer.texts_to_sequences(docs)
	# pad sequences
	padded = pad_sequences(encoded, maxlen=max_length, padding='post')
	return padded

Ahora estamos listos para definir nuestro modelo de red neuronal. 
1. El modelo utilizará una capa `Embedding` como la primera capa oculta. 
2. La capa `Embedding` requiere la especificación del tamaño del vocabulario, el tamaño del espacio vectorial de valor real y la longitud máxima de los documentos de entrada. 
3. El tamaño del vocabulario es el número total de palabras en nuestro vocabulario, más una para las palabras desconocidas.
4. Esta podría ser la longitud del conjunto de vocabulario o el tamaño del vocabulario dentro del `Tokenizer` utilizado para codificar los documentos con números enteros, por ejemplo:

In [17]:
# load training data
train_docs, ytrain = load_clean_dataset(vocab, True)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)

Vocabulary size: 25768


1. Usaremos un espacio vectorial de 100 dimensiones, pero puede probar otros valores, como 50 o 150. 
2. La longitud máxima del documento se calculó anteriormente en la variable de longitud máxima utilizada durante el relleno. 
3. La definición completa del modelo se enumera a continuación, incluida la capa `Embedding`. 
4. Utilizamos una red neuronal convolucional (CNN) ya que han demostrado ser exitosos en los problemas de clasificación de documentos. 
5. Se utiliza una configuración CNN con 32 filtros (campos paralelos para procesar palabras) y un tamaño de kernel de 8 con una función de activación `relu`. 
6. A esto le sigue una capa de agrupación que reduce a la mitad la salida de la capa convolucional.
7. A continuación, la salida 2D de la parte CNN del modelo se aplana en un vector 2D largo para representar las características extraídas por CNN. 
8. El back-end del modelo es un perceptrón multicapa estándar para interpretar las características de CNN. 
9. La capa de salida utiliza una función de activación `sigmoid` para generar un valor entre 0 y 1 para el negativo y positivo en la reseña.

In [18]:
# define the model
def define_model(vocab_size, max_length):
	model = Sequential()
	model.add(Embedding(vocab_size, 100, input_length=max_length))
	model.add(Conv1D(32, 8, activation='relu'))
	model.add(MaxPooling1D())
	model.add(Flatten())
	model.add(Dense(10, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# compile network
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize defined model
	model.summary()
	plot_model(model, to_file='model.png', show_shapes=True)
	return model

Podemos ver que la capa `Embedding` espera documentos con una longitud de 1317 palabras como entrada y codifica cada palabra del documento como un vector de 100 elementos y el tamaño del vocabulario de 25768.

In [19]:
model = define_model(vocab_size, max_length)

2022-12-05 16:48:19.793111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-05 16:48:20.404286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-05 16:48:20.405438: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-05 16:48:20.724203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1317, 100)         2576800   
                                                                 
 conv1d (Conv1D)             (None, 1310, 32)          25632     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 655, 32)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 20960)             0         
                                                                 
 dense (Dense)               (None, 10)                209610    
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                        

A continuación, se guarda un gráfico del modelo definido en un archivo con el nombre `model.png`.
  <img src="data/model_52.png" width="200" height="200" alt="CNN NLP">
  
A continuación, ajustamos la red a los datos de entrenamiento. 

In [ ]:
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
# define model
model = define_model(vocab_size, max_length)
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)

Una vez ajustado el modelo, se guarda en un archivo denominado `model.h5` para su posterior evaluación.

In [ ]:
# save the model
model.save('model.h5')

Ejecutar el ejemplo primero proporcionará un resumen del vocabulario del conjunto de datos de entrenamiento (25,768) y la longitud máxima de la secuencia de entrada en palabras (1,317). El ejemplo debería ejecutarse en unos minutos y el modelo de ajuste se guardará en un archivo.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section4"></a>
# <font color="#004D7F" size=6>4. Evaluar modelo</font>

En esta sección, evaluaremos el modelo entrenado y lo usaremos para hacer predicciones sobre nuevos datos. 

Podemos usar la función integrada `evaluate()` para estimar la habilidad del modelo en el conjunto de datos de train y test. Esto requiere que carguemos y codifiquemos los conjuntos de datos de train y test.

In [ ]:
# load all reviews
train_docs, ytrain = load_clean_dataset(vocab, True)
test_docs, ytest = load_clean_dataset(vocab, False)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
Xtest = encode_docs(tokenizer, max_length, test_docs)

Luego podemos cargar el modelo y evaluarlo en ambos conjuntos de datos e imprimir el accuracy.

In [ ]:
from keras.models import load_model
# load the model
model = load_model('model.h5')
# evaluate model on training dataset
_, acc = model.evaluate(Xtrain, ytrain, verbose=0)
print('Train Accuracy: %.2f' % (acc*100))
# evaluate model on test dataset
_, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %.2f' % (acc*100))

Luego, se deben preparar nuevos datos usando la misma codificación de texto y esquemas de codificación que se usaron en el conjunto de datos de entrenamiento. 

Una vez preparado, se puede hacer una predicción llamando a la función `predict()` en el modelo. La siguiente función llamada `predict_sentiment()` codificará y rellenará un texto de reseña de película dado y devolverá una predicción en términos de porcentaje y una etiqueta.

In [ ]:
# classify a review as negative or positive
def predict_sentiment(review, vocab, tokenizer, max_length, model):
	# clean review
	line = clean_doc(review, vocab)
	# encode and pad review
	padded = encode_docs(tokenizer, max_length, [line])
	# predict sentiment
	yhat = model.predict(padded, verbose=0)
	# retrieve predicted percentage and label
	percent_pos = yhat[0,0]
	if round(percent_pos) == 0:
		return (1-percent_pos), 'NEGATIVE'
	return percent_pos, 'POSITIVE'

Podemos probar este modelo con dos reseñas de películas ad hoc. 

In [ ]:
# test positive text
text = 'Everyone will enjoy this film. I love it, recommended!'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))
# test negative text
text = 'This is a bad movie. Do not watch it. It sucks.'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section5"></a>
# <font color="#004D7F" size=6>5. Desarrollar un modelo CNN para n-gramas</font>

El modelo se puede expandir usando múltiples redes neuronales convolucionales paralelas que leen el documento fuente usando diferentes tamaños de kernel. 

Esto, en efecto, crea una red neuronal convolucional multicanal para texto que lee texto con diferentes **tamaños de n-gramas (grupos de palabras)**.

Trabajamos con las mismas funciones pero sin tener que pasarle `vocab` como argumento.

In [30]:
from nltk.corpus import stopwords
import string
import re

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# prepare regex for char filtering
	re_punc = re.compile('[%s]' % re.escape(string.punctuation))
	# remove punctuation from each word
	tokens = [re_punc.sub('', w) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

# load the document
filename = 'data/txt_sentoken/pos/cv000_29590.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

['films', 'adapted', 'comic', 'books', 'plenty', 'success', 'whether', 'theyre', 'superheroes', 'batman', 'superman', 'spawn', 'geared', 'toward', 'kids', 'casper', 'arthouse', 'crowd', 'ghost', 'world', 'theres', 'never', 'really', 'comic', 'book', 'like', 'hell', 'starters', 'created', 'alan', 'moore', 'eddie', 'campbell', 'brought', 'medium', 'whole', 'new', 'level', 'mid', 'series', 'called', 'watchmen', 'say', 'moore', 'campbell', 'thoroughly', 'researched', 'subject', 'jack', 'ripper', 'would', 'like', 'saying', 'michael', 'jackson', 'starting', 'look', 'little', 'odd', 'book', 'graphic', 'novel', 'pages', 'long', 'includes', 'nearly', 'consist', 'nothing', 'footnotes', 'words', 'dont', 'dismiss', 'film', 'source', 'get', 'past', 'whole', 'comic', 'book', 'thing', 'might', 'find', 'another', 'stumbling', 'block', 'hells', 'directors', 'albert', 'allen', 'hughes', 'getting', 'hughes', 'brothers', 'direct', 'seems', 'almost', 'ludicrous', 'casting', 'carrot', 'top', 'well', 'anythi

In [31]:
# load all docs in a directory
def process_docs(directory, is_train):
	documents = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_train and filename.startswith('cv9'):
			continue
		if not is_train and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load the doc
		doc = load_doc(path)
		# clean doc
		tokens = clean_doc(doc)
		# add to list
		documents.append(tokens)
	return documents

In [32]:
# load and clean a dataset
def load_clean_dataset(is_train):
	# load documents
	neg = process_docs('data/txt_sentoken/neg', is_train)
	pos = process_docs('data/txt_sentoken/pos', is_train)
	docs = neg + pos
	# prepare labels
	labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
	return docs, labels

Guardamos los conjuntos train y test preparados en un archivo para que podamos cargarlos más tarde para el modelado y la evaluación del modelo. 

La función debajo llamada `save_dataset()` guardará un conjunto de datos preparado dado (elementos `X` e `y`) en un archivo usando la API `pickle` (esta es la API estándar para guardar objetos en Python).

In [33]:
# save a dataset to file
def save_dataset(dataset, filename):
	dump(dataset, open(filename, 'wb'))
	print('Saved: %s' % filename)

Ahora podemos llamar a las funciones para ejecutar nuestro ejemplo y que queden los corpus limpios.

In [34]:
import string
import re
from os import listdir
from nltk.corpus import stopwords
from pickle import dump

# load and clean all reviews
train_docs, ytrain = load_clean_dataset(True)
test_docs, ytest = load_clean_dataset(False)
# save training datasets
save_dataset([train_docs, ytrain], 'train.pkl')
save_dataset([test_docs, ytest], 'test.pkl')

Saved: train.pkl
Saved: test.pkl


<a id="section51"></a>
# <font color="#004D7F" size=5>5.1. Codificar datos</font>

El primer paso es cargar el conjunto de datos de entrenamiento limpio. Se puede llamar a la función denominada `load_dataset()` a continuación para cargar el conjunto de datos de entrenamiento en pickle.

In [35]:
# load a clean dataset
def load_dataset(filename):
	return load(open(filename, 'rb'))

- Debemos colocar un `Tokenizador` de Keras en el conjunto de datos de entrenamiento. 
- Usaremos este tokenizador para definir el vocabulario de la capa `Embedding` y codificar los documentos de revisión como números enteros. 
- La función `create_tokenizer()` a continuación creará un `Tokenizer` dada una lista de documentos.

In [36]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

También necesitamos saber la longitud máxima de las secuencias de entrada como entrada para el modelo y rellenar todas las secuencias a la longitud fija. 

La función `max_length()` calculará la longitud máxima (número de palabras) para todas las revisiones en el conjunto de datos de entrenamiento.

In [37]:
# calculate the maximum document length
def max_length(lines):
    #return max([len(s.split()) for s in lines])
    return max([len(s) for s in lines])

También necesitamos saber el tamaño del vocabulario para la capa `Embedding`. Esto se puede calcular a partir del `Tokenizer` preparado, de la siguiente manera:
```python
    # calculate vocabulary size
    vocab_size = len(tokenizer.word_index) + 1
```

- Finalmente, podemos codificar enteros y rellenar el texto limpio de la reseña de la película. 
- La función de abajo `encode_text()` codificará y rellenará los datos de texto a la longitud máxima.

In [38]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

<a id="section52"></a>
# <font color="#004D7F" size=5>5.2. Definir el modelo</font>

En Keras, se puede definir un modelo de entrada múltiple utilizando la API funcional. Definiremos un modelo con tres canales de entrada para procesar 4-gramas, 6-gramas y 8-gramas de texto. Cada canal se compone de los siguientes elementos:
- Capa `Input` que define la longitud de las secuencias de entrada.
- Capa `Embedding` ajustada al tamaño del vocabulario y representaciones de valor real de 100 dimensiones.
- Capa `Conv1D` con 32 filtros y un tamaño de kernel establecido en la cantidad de palabras para leer a la vez.
- Capa `MaxPooling1D` para consolidar la salida de la capa convolucional.
- Capa `Flatten` para reducir la salida tridimensional a bidimensional para la concatenación.
- La salida de los tres canales se concatena en un solo vector y se procesa mediante una capa `Dense` y una capa de salida. 

La siguiente función define y devuelve el modelo. 

In [39]:
# define the model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
	conv1 = Conv1D(32, 4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D()(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(32, 6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D()(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(32, 8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D()(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	model.summary()
	plot_model(model, show_shapes=True, to_file='model.png')
	return model

A continuación, se guarda un gráfico del modelo definido en un archivo con el nombre `model.png`.
  <img src="data/model_53.png" width="500" height="500" alt="CNN NLP">

<a id="section53"></a>
# <font color="#004D7F" size=5>5.3. Mostrar los resultados</font>

Ahora podemos ver el impacto de nuestra red neurnoal

In [40]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import concatenate

# load training dataset
trainLines, trainLabels = load_dataset('train.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length    
length = max_length(trainLines)
print('Max document length: %d' % length)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX], array(trainLabels), epochs=7, batch_size=16)
# save the model
model.save('model.h5')

Max document length: 1380
Vocabulary size: 44277


2022-12-05 17:03:16.602168: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.89MiB (rounded to 17710848)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-12-05 17:03:16.602275: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2022-12-05 17:03:16.602327: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 21, Chunks in use: 21. 5.2KiB allocated for chunks. 5.2KiB in use in bin. 304B client-requested in use in bin.
2022-12-05 17:03:16.602375: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-12-05 17:03:16.602417: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bi

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]

<a id="section54"></a>
# <font color="#004D7F" size=5>5.4. Evaluar el modelo</font>

Podemos evaluar el modelo de ajuste al predecir la reseña en todas las revisiones en el conjunto de datos de prueba no etiquetados

In [ ]:
from pickle import load
from keras.models import load_model
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# load datasets
trainLines, trainLabels = load_dataset('train.pkl')
testLines, testLabels = load_dataset('test.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
print('Max document length: %d' % length)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
testX = encode_text(tokenizer, testLines, length)
# load the model
model = load_model('model.h5')

Una vez cargado el modelo guardado podemos evaluarlo tanto en los conjuntos de datos de entrenamiento como de prueba. 

In [ ]:
# evaluate model on training dataset
_, acc = model.evaluate([trainX,trainX,trainX], array(trainLabels), verbose=0)
print('Train Accuracy: %.2f' % (acc*100))
# evaluate model on test dataset dataset
_, acc = model.evaluate([testX,testX,testX], array(testLabels), verbose=0)
print('Test Accuracy: %.2f' % (acc*100))

Ejecutar el ejemplo imprime la habilidad del modelo en los conjuntos de datos de entrenamiento y prueba. Podemos ver que, como era de esperar, la habilidad en el conjunto de datos de entrenamiento es excelente, aquí con una precisión del 100%.

También podemos ver que la habilidad del modelo en el conjunto de datos de prueba no etiquetados también es muy impresionante, logrando un 88,5%, que está por encima de la habilidad del modelo informado en el documento de 2014.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section6"></a>
# <font color="#004D7F" size=6>6. Extensiones</font>

<a id="section61"></a>
# <font color="#004D7F" size=5>6.1. Extensiones sin n-gramas</font>

En esta sección se enumeran algunas ideas para ampliar el tutorial que tal vez desee explorar.
- __Limpieza de datos__. Explore una mejor limpieza de datos, tal vez dejando algunos signos de puntuación intactos o normalizando las contracciones.
- __Secuencias truncadas__. Rellenar todas las secuencias con la duración de la secuencia más larga puede ser extremo si la secuencia más larga es muy diferente a todas las demás revisiones. Estudie la distribución de la duración de las reseñas y trunque las reseñas a una duración media.
- __Vocabulario truncado__. Eliminamos las palabras que aparecían con poca frecuencia, pero aún teníamos un gran vocabulario de más de 25,000 palabras. Explore aún más la reducción del tamaño del vocabulario y el efecto en la habilidad del modelo.
- __Filtros y Tamaño del Kernel__. La cantidad de filtros y el tamaño del kernel son importantes para modelar la habilidad y no se ajustaron. Explore el tunning de estos dos parámetros de CNN.
- __Épocas y tamaño de lote__. El modelo parece ajustarse rápidamente al conjunto de datos de entrenamiento. Explore configuraciones alternativas de la cantidad de épocas de entrenamiento y el tamaño del lote y use el conjunto de datos de prueba como un conjunto de validación para elegir un mejor punto de parada para entrenar el modelo.
- __Red más profunda__. Explore si una red más profunda da como resultado una mejor habilidad, ya sea en términos de capas CNN, capas MLP y ambas.
- __Pre-entrenamiento de Embedding__. Explore el entrenamiento previo de una palabra Word2Vec incrustada en el - modelo y el impacto en la habilidad del modelo con y sin ajustes adicionales durante el entrenamiento.
- __Utilice el Embedding de _GloVe___. Explore la carga de la incrustación de GloVe previamente entrenada y el impacto en la habilidad del modelo con y sin más ajustes durante el entrenamiento.
- __Revisiones de prueba más largas__. Explore si la habilidad de las predicciones del modelo depende de la duración de las reseñas de películas, como se sospecha en la sección final sobre la evaluación del modelo.
- __Modelo Final Entrenado__. Entrene un modelo final con todos los datos disponibles y utilícelo para hacer predicciones sobre reseñas de películas ad hoc reales de Internet.

<a id="section62"></a>
# <font color="#004D7F" size=5>6.2. Extensiones con n-gramas</font>

En esta sección se enumeran algunas ideas para ampliar el tutorial que tal vez desee explorar.
- __Diferentes n-gramas__. Explore el modelo cambiando el tamaño del kernel (número de n-gramas) que utilizan los canales en el modelo para ver cómo afecta la habilidad del modelo.
- __Más o menos canales__. Explore el uso de más o menos canales en el modelo y vea cómo afecta la habilidad del modelo.
- __Incrustación compartida__. Explore configuraciones donde cada canal comparte la misma palabra incrustada e informe sobre el impacto en la habilidad del modelo.
- __Red más profunda__. Las redes neuronales convolucionales funcionan mejor en visión artificial cuando son más profundas. Explore el uso de modelos más profundos aquí y vea cómo afecta la habilidad del modelo.
- __Secuencias truncadas__. Rellenar todas las secuencias con la duración de la secuencia más larga puede ser extremo si la secuencia más larga es muy diferente a todas las demás revisiones. Estudie la distribución de la duración de las reseñas y trunque las reseñas a una duración media.
- __Vocabulario truncado__. Eliminamos las palabras que aparecían con poca frecuencia, pero aún teníamos un gran vocabulario de más de 25,000 palabras. Explore aún más la reducción del tamaño del vocabulario y el efecto en la habilidad del modelo.
- __Épocas y tamaño de lote__. El modelo parece ajustarse rápidamente al conjunto de datos de entrenamiento. Explore configuraciones alternativas de la cantidad de épocas de entrenamiento y el tamaño del lote y use el conjunto de datos de prueba como un conjunto de validación para elegir un mejor punto de parada para entrenar el modelo.
- __Pre-entrenamiento e incrustación__. Explore el entrenamiento previo de una palabra Word2Vec incrustada en el modelo y el impacto en la habilidad del modelo con y sin ajustes adicionales durante el entrenamiento.
- __Utilice la incrustación de GloVe__. Explore la carga de la incrustación de GloVe previamente entrenada y el impacto en la habilidad del modelo con y sin más ajustes durante el entrenamiento.
- __Entrenamiento del Modelo Final__. Entrene un modelo final con todos los datos disponibles y utilícelo para hacer predicciones sobre reseñas de películas ad hoc reales de Internet.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>